In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Layer,Conv2D,Flatten,Dense,Input,MaxPooling2D
from keras.models import Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
import tensorflow as tf

In [2]:
datagen=ImageDataGenerator(
rescale=1.0/255,
# rotation_range=40,
# height_shift_range=0.2,
# width_shift_range=0.2, 
# zoom_range=0.2, 
# fill_mode="nearest",
# horizontal_flip=True,
# shear_range=0.2,
validation_split=0.2
)

In [3]:
df="/kaggle/input/clouds-gan-dataset/"
train_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
subset="training")
val_data=datagen.flow_from_directory(
df,
target_size=(256,256),
batch_size=32,
subset="validation")

Found 1303 images belonging to 1 classes.
Found 325 images belonging to 1 classes.


In [4]:
!pip install cirq

In [5]:
import cirq

In [6]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(4)]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*2*np.pi).on(qubit[i%len(qubit)]))
        circuit.append(cirq.CNOT(qubit[i % len(qubit)], qubit[(i + 1) % len(qubit)]))
        circuit.append(cirq.measure(qubit[i%len(qubit)],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        output_flat=tf.reshape(inputs,[-1,256*256*3])
        output=tf.numpy_function(run_quantum_circuit,[output_flat],tf.float32)
        output.set_shape((inputs.shape[0],inputs.shape[1]))
        return output
model=Sequential([
#     Input(shape=(256,256,3)),
#     Conv2D(32,(3,3),activation='relu',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
#     MaxPooling2D(pool_size=(4,4)),
#     Flatten(),
    QuantumLayer(),
    Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
model.fit(train_data,epochs=5,batch_size=32)

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
